## Anonymous Walks for Graph Embeddings
(based on the paper [Anonymous Walk Embeddings](https://arxiv.org/abs/1805.11921) 2018, by Sergey Ivanov and Evgeny Burnaev)

### What are graph embeddings ?

Graph embeddings are a way to represent the nodes and edges of a graph in a lower-dimensional vector space. Graphs are complex data structures that consist of nodes and edges. They are used to model relationships between entities in various domains such as social networks, recommendation systems, biology, and knowledge graphs.

Graph embeddings aim to capture the structural and relational information of the graph in a continuous vector space. This representation makes it easier to perform various machine learning tasks such as node classification, link prediction, and graph clustering.

![Graph embeddings](images/embeddings.jpg)

### What are anonymous walks ?

In order to understand anonymous walks, we first need to understand random walks. 

Thus, a random walk is a mathematical formalization of a path that consists of a succession of random steps. In the context of graph theory, a random walk on a graph is a traversal that begins at a starting node and moves to a neighboring node at each step based on a probability distribution.

<img src="images/random_walk.png" alt="Random walk" width="500"/>

For example, a random walk on the graph above might be: <span style="color:red; font-weight: bold;">1 -> 3 -> 4 -> 3 -> 2</span>

Normally states in a random walk correspond to a label or a global name of a node; however, for certain reasons, such as privacy and security, such states could be unavailable. It has been shown that an anonymized version of a random walk can provide a flexible way to reconstruct a network even when global names are absent. We next define the notion of **anonymous walk**.

<span style="color:red; font-weight: bold;">Definition 1: </span> An anonymous walk is a type of walk on a graph where only the underlying structure of the walk is considered, ignoring the specific identities of the nodes. This abstraction provides a robust and efficient way to capture graph structures, especially in settings where node identities are irrelevant or unavailable.

<span style="color:red; font-weight: bold;">Definition 2: </span> . Let s = (u<sub>1</sub>, u<sub>2</sub> , ... , u<sub>k</sub>) be an ordered list of elements u<sub>i</sub> ∈ V. We define the positional function pos: (s, u<sub>i</sub>) -> q such that for any ordered list s = (u<sub>1</sub>, u<sub>2</sub> , ... , u<sub>k</sub>) and an element u<sub>i</sub> ∈ V it returns a list q =  (p<sub>1</sub>, p<sub>2</sub> , ... , p<sub>l</sub>) for all positions p<sub>j</sub> ∈ N of u<sub>i</sub> occurences in a list s.

For example, if  s = (a, b, c, b, c) then pos(s, a) = 1 as element a appears only on the first position and pos(s, b) = (2, 4).

We denote mapping of a random walk **w** to anonymous walk **a** by w → a.


<img src="images/mapping.png" alt="Random walk" width="800"/>

In the image above we can see an example demonstrating the concept of anonymous
walk. Two different random walks 1 and 2 of the graph correspond
to the same anonymous walk 1. A random walk 3 corresponds to
another anonymous walk 2.

### How do we use anonymous walks to generate embeddings ?

The question remains: How do we use these anonymous walks to generate embeddings for a graph ?
In their paper, Ivanov and Burnaev propose 2 unsupervised algorithms for generating embeddings. The first algorithm is feature-based, while the second is data-driven.\
Feature-based approaches are those where the model is explicitly provided with engineered features that are designed based on domain knowledge or insight into the data. Data-driven approaches, also known as end-to-end or representation learning approaches, are those where the model is provided with raw data and it automatically learns useful features during the training process.
 - <span style="color:red; font-weight: bold;">Feature-based embedding</span>  Anonymous walk embedding of a graph G is the vector f<sub>G</sub> of size n, whose i-th component corresponds to a probability p(a<sub>i</sub>) of having anonymous walk a<sub>i</sub> in a graph G.\
 f<sub>G</sub> = (p(a<sub>1</sub>), p(a<sub>2</sub>), ... , p(a<sub>n</sub>),) i.e. the values in the embedding vector are the probabilities with which certain anonymous walks appear in the graph.\
 However, as the exact calculation of network embeddings can be computationally expensive, Ivanov and Burnaev demonstrate a way to sample walks in graph, in order to approximate the actual distribution with a given confidence.
 - <span style="color:red; font-weight: bold;">Data-driven embedding</span> Next, they show how one can learn
distributed graph representations in a data-driven manner,
similar to learning paragraph vectors in NLP. In this case, an anonymous walk is a
word, a randomly sampled set of anonymous walks starting
from the same node is a set of co-occurring words, and a
graph is a document.
In their experiments, they show that network embeddings
even with simple SVM classifier achieve an increase in classification accuracy compared to state-of-the-art supervised
neural network methods and graph kernels. This demonstrates that representation of your data can be more promising subject to study than the type and architecture of your
predictive model.

### Code

We first want to read the graphs stored and transform them into a **networkx** Digraph object.

In [31]:
from src.AnonymousWalkKernel import GraphKernel
from src.AnonymousWalkKernel import AnonymousWalks
import time
import numpy as np

graph_kernel = GraphKernel()
graph_kernel.read_graphs(folder="Datasets/mutag/", ext='graphml')

print('Number of graphs: ', len(graph_kernel.graphs))

Number of graphs:  188


#### Feature based embeddings 

For feature based embeddings, we want to calculate the distribution of anonymous walks in a graph.

In [32]:
# We will calculate all of the anonymous walks of length 4 for the first graph in the dataset
G = graph_kernel.graphs[0]
anonymous_walk = AnonymousWalks(G)
length = 4

anonymous_walk._all_paths(steps=length)
print('All possible anonymous walks of length {}'.format(length))
print(anonymous_walk.paths[length])

All possible anonymous walks of length 4
[[0, 1, 0], [0, 1, 2], [0, 1, 2, 0], [0, 1, 0, 1], [0, 1, 2, 1], [0, 1, 0, 2], [0, 1, 2, 3], [0, 1, 0, 1, 0], [0, 1, 2, 1, 0], [0, 1, 0, 2, 0], [0, 1, 2, 3, 0], [0, 1, 2, 0, 1], [0, 1, 0, 2, 1], [0, 1, 2, 3, 1], [0, 1, 2, 0, 2], [0, 1, 0, 1, 2], [0, 1, 2, 1, 2], [0, 1, 2, 3, 2], [0, 1, 2, 0, 3], [0, 1, 2, 1, 3], [0, 1, 0, 2, 3], [0, 1, 2, 3, 4]]


Let's try calculating all of the anonymous walks for a larger length

In [33]:
length = 13

start_time = time.time()
anonymous_walk._all_paths(steps=length)
end_time = time.time()

print('Number of possible anonymous walks for length {} = {} walks'.format(length, len(anonymous_walk.paths[length])))
print('Time taken to calculate all possible anonymous walks of length {}: {} seconds'.format(length, end_time - start_time))

Number of possible anonymous walks for length 13 = 32679020 walks
Time taken to calculate all possible anonymous walks of length 13: 66.34404420852661 seconds


We can get _exact_ distribution of anonymous walks in a graph for small lengths (e.g. $l <= 7$) using the `embed` method.

In [34]:
def print_anonymous_walk_embeddings(walks, embeddings):
    print('Walk\t\t\tEmbedding')    
    for i, walk in enumerate(walks):
        print("{}\t{}".format(walk, embeddings[i]))
        print()

In [35]:
length = 4
start = time.time()
embedding, meta = anonymous_walk.embed(steps = length, method = 'exact', keep_last=True, verbose=False)
finish = time.time()

aws = meta['meta-paths']
print('Computed Embedding with a length of {} in {:.3f} sec.'.format(len(aws), finish - start))
print()
print_anonymous_walk_embeddings(embeddings=embedding, walks=aws)

Computed Embedding with a length of 15 in 0.003 sec.

Walk			Embedding
[0, 1, 0, 1, 0]	0.08742619431025227

[0, 1, 2, 1, 0]	0.09923510466988726

[0, 1, 0, 2, 0]	0.09923510466988733

[0, 1, 2, 3, 0]	0

[0, 1, 2, 0, 1]	0

[0, 1, 0, 2, 1]	0

[0, 1, 2, 3, 1]	0

[0, 1, 2, 0, 2]	0

[0, 1, 0, 1, 2]	0.10782340311325822

[0, 1, 2, 1, 2]	0.09923510466988726

[0, 1, 2, 3, 2]	0.13560118089103612

[0, 1, 2, 0, 3]	0

[0, 1, 2, 1, 3]	0.0539452495974235

[0, 1, 0, 2, 3]	0.12822061191626416

[0, 1, 2, 3, 4]	0.18927804616210467



We can also use the _sampling_ approach to get the distribution, based on this formula:

<img src="images/prob_formula.png" alt="Formula" width="500"/>

Where **m** is the number of samples needed in order to get within a certain margin of error to the real distribution. The margin or error is controlled by $\varepsilon$ and $\delta$.\
The probability that the difference between the real distribution and the sampled distribution is bigger than $\varepsilon$ should be smaller than $\delta$

<img src="images/prob_formula2.png" alt="Formula" width="500"/>

In [36]:
length = 8
start = time.time()
embedding, meta = anonymous_walk.embed(steps = length, method = 'sampling', keep_last=True, verbose=False, eps=0.1, delta=0.01)
finish = time.time()


aws = meta['meta-paths']
print('Computed Embedding with a length of {} in {:.3f} sec.'.format(len(aws), finish - start))
print()
print_anonymous_walk_embeddings(embeddings=embedding, walks=aws)

Computed Embedding with a length of 4140 in 17.491 sec.

Walk			Embedding
[0, 1, 0, 1, 0, 1, 0, 1, 0]	0.005236175886800904

[0, 1, 2, 1, 0, 1, 0, 1, 0]	0.004655151718631077

[0, 1, 0, 2, 0, 1, 0, 1, 0]	0.004453358893757724

[0, 1, 2, 3, 0, 1, 0, 1, 0]	0

[0, 1, 2, 0, 2, 1, 0, 1, 0]	0

[0, 1, 0, 1, 2, 1, 0, 1, 0]	0.004509025879929684

[0, 1, 2, 1, 2, 1, 0, 1, 0]	0.004100221450229357

[0, 1, 2, 3, 2, 1, 0, 1, 0]	0.0038323240792767837

[0, 1, 2, 0, 3, 1, 0, 1, 0]	0

[0, 1, 2, 1, 3, 1, 0, 1, 0]	0.0014421228605176802

[0, 1, 0, 2, 3, 1, 0, 1, 0]	0

[0, 1, 2, 3, 4, 1, 0, 1, 0]	0

[0, 1, 0, 1, 0, 2, 0, 1, 0]	0.004519463439836926

[0, 1, 2, 1, 0, 2, 0, 1, 0]	0

[0, 1, 0, 2, 0, 2, 0, 1, 0]	0.004107179823500852

[0, 1, 2, 3, 0, 2, 0, 1, 0]	0

[0, 1, 2, 0, 1, 2, 0, 1, 0]	0

[0, 1, 0, 2, 1, 2, 0, 1, 0]	0

[0, 1, 2, 3, 1, 2, 0, 1, 0]	0

[0, 1, 2, 0, 3, 2, 0, 1, 0]	0

[0, 1, 2, 1, 3, 2, 0, 1, 0]	0

[0, 1, 0, 2, 3, 2, 0, 1, 0]	0.003656625154171493

[0, 1, 2, 3, 4, 2, 0, 1, 0]	0

[0, 1, 2, 1, 0, 3, 0,

#### Getting a kernel matrix from embeddings

First, we must compute the feature matrix, for which we must embed all of our graphs.

In [37]:
length = 5
graph_kernel.embed_graphs(steps = length, keep_last=True)

embeddings = graph_kernel.embeddings
print('Embeddings matrix of shape: {} x {}'.format(embeddings.shape[0], embeddings.shape[1]))
print(embeddings[:10, :])

Using exact method to get graph embeddings
Dimension size: 52
Processing 0 graph
Processing 10 graph
Processing 20 graph
Processing 30 graph
Processing 40 graph
Processing 50 graph
Processing 60 graph
Processing 70 graph
Processing 80 graph
Processing 90 graph
Processing 100 graph
Processing 110 graph
Processing 120 graph
Processing 130 graph
Processing 140 graph
Processing 150 graph
Processing 160 graph
Processing 170 graph
Processing 180 graph
Embeddings matrix of shape: 188 x 52
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.04353417
  0.04908078 0.04062668 0.         0.         0.04908078 0.04908078
  0.0527487  0.         0.02388621 0.         0.         0.04389202
  0.         0.04062668 0.         0.         0.         0.
  0.         0.         0.0527487  0.         0.05015432 0.01798175
  0.         0.         0.         0.         0.         0.05874262
  0.05015432 0.0527487  0.08038111 0.         0.     

Now we must compute the kernel matrix. We can use three types of kernels: RBF, Polynomial or Dot.

In [38]:
graph_kernel.embeddings = embeddings
graph_kernel.kernel_matrix(kernel_method='rbf', build_embeddings=False)
matrix = graph_kernel.K
print('Kernel matrix has shape: {} x {}'.format(matrix.shape[0], matrix.shape[1]))
print(matrix[:10, :])

Kernel matrix has shape: 188 x 188
[[1.         0.99989996 0.99999765 ... 0.99995036 0.99996831 0.99994935]
 [0.99989996 1.         0.99992404 ... 0.99997508 0.99995131 0.99996084]
 [0.99999765 0.99992404 1.         ... 0.9999645  0.99997687 0.99996336]
 ...
 [0.99999843 0.99991782 0.99999844 ... 0.99996579 0.9999801  0.99996494]
 [0.99999529 0.99992904 0.99999718 ... 0.99995993 0.99996784 0.99995399]
 [0.99997598 0.99996699 0.99998727 ... 0.99999234 0.99998998 0.99998984]]


#### Performance

In [39]:
y = []
with open('Datasets/mutag/labels.txt') as f:
    for line in f:
        y.extend(list(map(int, line.strip().split())))
y = np.array(y)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [74]:
from src.AnonymousWalkKernel import Evaluation
k = 10
ev = Evaluation(matrix = matrix, labels = y, verbose = False)
accuracies = ev.evaluate(k=k)
print('Performed {}-fold validation and scored {:.2f}% mean accuracy in graph classification'.format(k, 100*np.mean(accuracies)))

Performed 10-fold validation and scored 66.54% mean accuracy in graph classification


#### Data-driven embeddings

In [75]:

dataset = 'mutag'

batch_size = 100
window_size = 8
neighborhood_size = 8
embedding_size_w = 128
embedding_size_d = 128
num_samples = 32

concat = False
loss_type = 'sampled_softmax'
optimize = 'Adagrad'
learning_rate = 1.0
root = 'Datasets/'
ext = 'graphml'
steps = 7
epochs = 1
batches_per_epoch = 50
candidate_func = None
graph_labels = None

In [79]:
from src.AnonymousWalkEmbeddings import AWE

awe = AWE(dataset = dataset, batch_size = batch_size, window_size = window_size,
                  embedding_size_w = embedding_size_w, embedding_size_d = embedding_size_d,
                  num_samples = num_samples, concat = concat, loss_type = loss_type,
                  optimize = optimize, learning_rate = learning_rate, root = root,
                  ext = ext, steps = steps, epochs = epochs, batches_per_epoch = batches_per_epoch,
                  candidate_func = candidate_func, graph_labels=graph_labels, neighborhood_size=neighborhood_size)

Number of graphs: 188
Generating corpus... Finished 0.0019996166229248047
Number of words: 877


In [84]:
print()
start2emb = time.time()
print(awe.nodes_per_graphs)
awe.train() # get embeddings
finish2emb = time.time()
print()
print('Time to compute embeddings: {:.2f} sec'.format(finish2emb - start2emb))


{}
Initialized
Epoch: 0


KeyError: 93